In [1]:
import pandas as pd
import warnings
import docx2txt
import os
import nltk
import glob


nltk.download('stopwords')
nltk.download('punkt')
pd.set_option('display.max_columns', None) #to see all columns in a wide DataFrame
warnings.filterwarnings('ignore')
%matplotlib inline

print('File Working On: ',os.getcwd())#get the path of current working directory
%run '/home/uday/Desktop/Resume Screening/functions.ipynb'

[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>
[nltk_data] Error loading punkt: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>


File Working On:  /home/uday/Desktop/Resume Screening/deployment


[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>
[nltk_data] Error loading punkt: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>


In [4]:
# Set the path to the directory containing the files
directory_path = r"/home/uday/Desktop/Resume Screening/Resume"

def preprocess(directory_path = directory_path):

    # Initialize an empty list to hold the text data
    text_data = []

    # Loop through each file in the directory
    for file_path in glob.glob(os.path.join(directory_path, "*.docx")):

        text_data.append(docx2txt.process(file_path))
        
    
    df = pd.DataFrame({"Resume": text_data})

    df['Resume'] = df['Resume'].str.replace('\n', ' ')
    df['Resume'] = df['Resume'].str.replace('\r', ',')
    df['Resume'] = df['Resume'].str.replace('\t', '')
    #getting the data with extrated features
    df[['name', 'email', 'skills']] = df['Resume'].apply(lambda x: pd.Series(extract_info(x)))

    #Cleaning the text on resume column
    df['cleaned_resume'] = df['Resume'].apply(clean_resume)

    #Cleaning skills
    df['skills'] = df['skills'].apply(clean_text)

    df = df.fillna('unknown')

    return df

In [5]:
data = preprocess()
data.head()

,Resume,name,email,skills,cleaned_resume
0,"Ranjan Kumar Verma (PMP, CSM) Project Manager...",Ranjan Kumar,unknown,technicalissues businessprocess certification ...,ranjan kumar verma pmp csm project manager scr...
1,"Suresh Basetti Agile Expert, Scrum Master, ...",Suresh Basetti,sureshkumar.basetti@gmail.com,installation linux architecture resourcemanage...,suresh basetti agile expert scrum master proje...
2,"Padmasri Dantu Bellevue, WA 98007 • padmasr...",Padmasri Dantu,padmasri.vamsi@gmail.com,c userstories updates sdlc shell java visual s...,padmasri dantu bellevue wa 98007 padmasri vams...
3,Srivatsan Ramabhadran sreevatsan77@gm...,Srivatsan Ramabhadran,sreevatsan77@gmail.com,userstories installation projectmanagement arc...,srivatsan ramabhadran sreevatsan77 gmail com 9...


In [6]:
#Save Model
import pickle

filename = 'SVM_model.sav'
load_model = pickle.load(open(filename, 'rb'))

In [7]:
# predict on the whole dataset using the trained pipeline model
predictions = load_model.predict(data['cleaned_resume'])

# add the predictions as a new column to the original dataframe
data['Prediction'] = predictions

data

,Resume,name,email,skills,cleaned_resume,Prediction
0,"Ranjan Kumar Verma (PMP, CSM) Project Manager...",Ranjan Kumar,unknown,technicalissues businessprocess certification ...,ranjan kumar verma pmp csm project manager scr...,project manager
1,"Suresh Basetti Agile Expert, Scrum Master, ...",Suresh Basetti,sureshkumar.basetti@gmail.com,installation linux architecture resourcemanage...,suresh basetti agile expert scrum master proje...,project manager
2,"Padmasri Dantu Bellevue, WA 98007 • padmasr...",Padmasri Dantu,padmasri.vamsi@gmail.com,c userstories updates sdlc shell java visual s...,padmasri dantu bellevue wa 98007 padmasri vams...,project manager
3,Srivatsan Ramabhadran sreevatsan77@gm...,Srivatsan Ramabhadran,sreevatsan77@gmail.com,userstories installation projectmanagement arc...,srivatsan ramabhadran sreevatsan77 gmail com 9...,project manager
